In [24]:
import os
import json
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import f1_score
from pathlib import Path

# === CONFIGURATION ===
img_size = (128, 128)
batch_size = 32
epochs = 50
num_classes = 5
l2_lambda = 0.01
k_folds = 5

base_dir = Path("splitted_data_kfold")
results = []

# === DATA AUGMENTATION ===
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    zoom_range=0.1
)
val_datagen = ImageDataGenerator(rescale=1./255)

In [25]:
# === MODEL BUILDER ===
def build_model():
    model = models.Sequential([
        # Layer 1
        layers.Conv2D(32, (3,3), activation='relu', input_shape=img_size + (3,)),
        layers.MaxPooling2D((2,2), strides=2),

        # Layer 2
        layers.Conv2D(64, (3,3), activation='relu'),
        layers.MaxPooling2D((2,2), strides=2),
        layers.Dropout(0.2),

        # Layer 3
        layers.Conv2D(128, (3,3), activation='relu'),
        layers.AveragePooling2D((2,2), strides=2),

        layers.Flatten(),
        layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(l2_lambda)),
        layers.Dense(num_classes, activation='softmax')
    ])

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

In [26]:
# === TRAIN & EVALUATE PER FOLD ===
for fold in range(1, k_folds + 1):
    print(f"\n🔹 Fold {fold}/{k_folds}")
    train_dir = base_dir / f"fold_{fold}" / "train"
    val_dir   = base_dir / f"fold_{fold}" / "val"

    # Generators per fold
    train_gen = train_datagen.flow_from_directory(
        train_dir, target_size=img_size, batch_size=batch_size, class_mode="categorical"
    )
    val_gen = val_datagen.flow_from_directory(
        val_dir, target_size=img_size, batch_size=batch_size, class_mode="categorical", shuffle=False
    )

    print("Class indices:", train_gen.class_indices)
    for cls, idx in train_gen.class_indices.items():
        print(cls, ":", sum(train_gen.classes == idx))


    model = build_model()
    callbacks = [
        EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    ]

    history = model.fit(
        train_gen,
        epochs=epochs,
        validation_data=val_gen,
        callbacks=callbacks,
        verbose=1
    )

    # Evaluate
    val_loss, val_acc = model.evaluate(val_gen, verbose=0)
    y_true = val_gen.classes
    y_pred = np.argmax(model.predict(val_gen, verbose=0), axis=1)
    f1 = f1_score(y_true, y_pred, average='macro')

    print(f"Fold {fold} Results: accuracy={val_acc:.4f}, loss={val_loss:.4f}, f1={f1:.4f}")
    results.append({'fold': fold, 'accuracy': val_acc, 'loss': val_loss, 'f1': f1})


🔹 Fold 1/5
Found 800 images belonging to 5 classes.
Found 200 images belonging to 5 classes.
Class indices: {'American_Bulldog': 0, 'German_Shorthaired': 1, 'Havanese': 2, 'Maine_Coon': 3, 'Pomeranian': 4}
American_Bulldog : 160
German_Shorthaired : 160
Havanese : 160
Maine_Coon : 160
Pomeranian : 160
Epoch 1/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 9s 298ms/step - accuracy: 0.2537 - loss: 2.3740 - val_accuracy: 0.2850 - val_loss: 1.7452
Epoch 2/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 7s 287ms/step - accuracy: 0.3525 - loss: 1.5739 - val_accuracy: 0.4300 - val_loss: 1.5436
Epoch 3/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 8s 303ms/step - accuracy: 0.4563 - loss: 1.4382 - val_accuracy: 0.4500 - val_loss: 1.5085
Epoch 4/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 8s 310ms/step - accuracy: 0.5163 - loss: 1.3498 - val_accuracy: 0.5050 - val_loss: 1.4911
Epoch 5/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 7s 296ms/step - accuracy: 0.5400 - loss: 1.3164 - val_accuracy: 0.4550 - val_loss: 1.4632
Epoch 6/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 7s 291ms/step - ac

c:\Users\ASUS\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\ASUS\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 9s 315ms/step - accuracy: 0.1963 - loss: 2.5348 - val_accuracy: 0.2000 - val_loss: 1.9202
Epoch 2/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 7s 295ms/step - accuracy: 0.2562 - loss: 1.7738 - val_accuracy: 0.3150 - val_loss: 1.6517
Epoch 3/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 8s 305ms/step - accuracy: 0.2950 - loss: 1.6146 - val_accuracy: 0.3650 - val_loss: 1.5619
Epoch 4/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 8s 298ms/step - accuracy: 0.3900 - loss: 1.4898 - val_accuracy: 0.4150 - val_loss: 1.4611
Epoch 5/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 7s 296ms/step - accuracy: 0.4300 - loss: 1.4343 - val_accuracy: 0.4650 - val_loss: 1.3768
Epoch 6/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 8s 299ms/step - accuracy: 0.5013 - loss: 1.3170 - val_accuracy: 0.4600 - val_loss: 1.4098
Epoch 7/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 8s 323ms/step - accuracy: 0.5325 - loss: 1.2557 - val_accuracy: 0.4850 - val_loss: 1.3701
Epoch 8/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 8s 328ms/step - accuracy: 0.5750 - loss: 1.1887 - val_accuracy: 0.

c:\Users\ASUS\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\ASUS\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 11s 363ms/step - accuracy: 0.2262 - loss: 2.4292 - val_accuracy: 0.1950 - val_loss: 1.8203
Epoch 2/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 8s 328ms/step - accuracy: 0.2237 - loss: 1.7052 - val_accuracy: 0.2950 - val_loss: 1.6362
Epoch 3/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 8s 319ms/step - accuracy: 0.2788 - loss: 1.5933 - val_accuracy: 0.3950 - val_loss: 1.4723
Epoch 4/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 8s 300ms/step - accuracy: 0.3775 - loss: 1.4912 - val_accuracy: 0.3900 - val_loss: 1.4500
Epoch 5/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 8s 327ms/step - accuracy: 0.4200 - loss: 1.4210 - val_accuracy: 0.4100 - val_loss: 1.4777
Epoch 6/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 8s 333ms/step - accuracy: 0.4487 - loss: 1.3851 - val_accuracy: 0.3800 - val_loss: 1.4950
Epoch 7/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 8s 326ms/step - accuracy: 0.4963 - loss: 1.3630 - val_accuracy: 0.4700 - val_loss: 1.3782
Epoch 8/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 8s 308ms/step - accuracy: 0.4650 - loss: 1.3562 - val_accuracy: 0

...

c:\Users\ASUS\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\ASUS\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 10s 350ms/step - accuracy: 0.2013 - loss: 2.6688 - val_accuracy: 0.2000 - val_loss: 1.9988
Epoch 2/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 8s 314ms/step - accuracy: 0.2837 - loss: 1.8135 - val_accuracy: 0.3100 - val_loss: 1.7040
Epoch 3/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 8s 319ms/step - accuracy: 0.3925 - loss: 1.5800 - val_accuracy: 0.4700 - val_loss: 1.5283
Epoch 4/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 8s 305ms/step - accuracy: 0.4575 - loss: 1.4243 - val_accuracy: 0.4800 - val_loss: 1.4027
Epoch 5/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 8s 309ms/step - accuracy: 0.5063 - loss: 1.3343 - val_accuracy: 0.4500 - val_loss: 1.5185
Epoch 6/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 7s 292ms/step - accuracy: 0.5437 - loss: 1.2630 - val_accuracy: 0.5000 - val_loss: 1.3669
Epoch 7/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 8s 305ms/step - accuracy: 0.5888 - loss: 1.2197 - val_accuracy: 0.4250 - val_loss: 1.5075
Epoch 8/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 8s 306ms/step - accuracy: 0.5975 - loss: 1.1788 - val_accuracy: 0

Fold 4 Results: accuracy=0.5650, loss=1.2286, f1=0.5695

🔹 Fold 5/5
Found 800 images belonging to 5 classes.
Found 200 images belonging to 5 classes.
Class indices: {'American_Bulldog': 0, 'German_Shorthaired': 1, 'Havanese': 2, 'Maine_Coon': 3, 'Pomeranian': 4}
American_Bulldog : 160
German_Shorthaired : 160
Havanese : 160
Maine_Coon : 160
Pomeranian : 160


c:\Users\ASUS\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\ASUS\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 10s 337ms/step - accuracy: 0.2113 - loss: 2.7230 - val_accuracy: 0.2000 - val_loss: 2.0020
Epoch 2/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 8s 301ms/step - accuracy: 0.2537 - loss: 1.8280 - val_accuracy: 0.2000 - val_loss: 1.7531
Epoch 3/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 8s 303ms/step - accuracy: 0.3150 - loss: 1.6473 - val_accuracy: 0.3000 - val_loss: 1.5988
Epoch 4/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 8s 300ms/step - accuracy: 0.3738 - loss: 1.5009 - val_accuracy: 0.3700 - val_loss: 1.4168
Epoch 5/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 7s 294ms/step - accuracy: 0.4425 - loss: 1.4442 - val_accuracy: 0.4500 - val_loss: 1.4641
Epoch 6/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 7s 296ms/step - accuracy: 0.4725 - loss: 1.3831 - val_accuracy: 0.5250 - val_loss: 1.2985
Epoch 7/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 7s 296ms/step - accuracy: 0.4750 - loss: 1.3557 - val_accuracy: 0.5200 - val_loss: 1.3256
Epoch 8/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 7s 293ms/step - accuracy: 0.5362 - loss: 1.2619 - val_accuracy: 0

In [ ]:
# === AVERAGE METRICS ===
accs = [r['accuracy'] for r in results]
losses = [r['loss'] for r in results]
f1s = [r['f1'] for r in results]

avg_acc = np.mean(accs)
avg_loss = np.mean(losses)
avg_f1 = np.mean(f1s)
std_acc = np.std(accs)
std_f1 = np.std(f1s)

print("\n📊 5-Fold Cross Validation Summary:")
print(f"Accuracy : {avg_acc:.4f} ± {std_acc:.4f}")
print(f"F1-Score : {avg_f1:.4f} ± {std_f1:.4f}")
print(f"Loss     : {avg_loss:.4f}")

# Save metrics
os.makedirs("checkpoints_exp2", exist_ok=True)
with open("checkpoints_exp2/exp2_kfold_results.json", "w") as f:
    json.dump(results, f, indent=2)

# === FINAL MODEL TRAINING ON FULL DATA ===
print("\n🔸 Training final model on all data with best configuration...")
final_train_dir = Path("splitted_data_kfold/fold_1/train")

final_train_gen = train_datagen.flow_from_directory(
    final_train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=False   # keep order for evaluation
)

final_model = build_model()
history_final = final_model.fit(
    final_train_gen,
    epochs=epochs,
    verbose=1
)
final_model.save("checkpoints_exp2/final_model.h5")

# === EVALUATE FINAL MODEL ON FULL TRAINING DATA ===
final_loss, final_acc = final_model.evaluate(final_train_gen, verbose=0)
y_true_full = final_train_gen.classes
y_pred_full = np.argmax(final_model.predict(final_train_gen, verbose=0), axis=1)
final_f1 = f1_score(y_true_full, y_pred_full, average='macro')

print("\n📘 Final Model (Full Training Data) Results:")
print(f"Accuracy: {final_acc:.4f}")
print(f"Loss:     {final_loss:.4f}")
print(f"F1-Score: {final_f1:.4f}")

# === COMBINE AND SAVE ALL RESULTS ===
summary = {
    "cross_validation": {
        "accuracy_mean": float(avg_acc),
        "accuracy_std": float(std_acc),
        "f1_mean": float(avg_f1),
        "f1_std": float(std_f1),
        "loss_mean": float(avg_loss)
    },
    "final_model": {
        "accuracy": float(final_acc),
        "loss": float(final_loss),
        "f1_score": float(final_f1)
    },
    "per_fold": results
}

os.makedirs("checkpoints_exp2", exist_ok=True)
with open("checkpoints_exp2/exp2_full_summary.json", "w") as f:
    json.dump(summary, f, indent=2)

print("\n✅ Experiment 2 complete. Summary saved to checkpoints_exp2/exp2_full_summary.json")


📊 5-Fold Cross Validation Summary:
Accuracy : 0.5160 ± 0.0469
F1-Score : 0.5052 ± 0.0574
Loss     : 1.3063

🔸 Training final model on all data with best configuration...
Found 800 images belonging to 5 classes.


c:\Users\ASUS\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\ASUS\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
